<a href="https://colab.research.google.com/github/AndreyKuratov/project_mldm_21/blob/main/MLDM_proj_4_simple_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Загрузка библиотек

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import soundfile as sf
import scipy.signal as signal
import os
from tqdm import tqdm

import os
import csv
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image

import librosa as lb 
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as ipd

from skimage.transform import resize
from scipy import stats

from sklearn.preprocessing import MinMaxScaler
from skimage.transform import resize

# 1. Загрузка данных и меток

In [ ]:
PATH_TO_DIR = '/content/drive/MyDrive/mldm21_project/'
PATH_TO_SAVE = '/content/drive/MyDrive/mldm21_project/'

data = np.load(PATH_TO_SAVE+'sr_48e3_nfft_2e11_hlop_2e9_pict_256x512_u8.npz',allow_pickle=True)
# 'imgs' 'labels'
labels = np.load(PATH_TO_SAVE+'labels_simple.npz')
# 'files_fp' 'labels_fp' 'files_tp' 'labels_tp'

labels_pd_fp = pd.read_csv('/content/drive/MyDrive/mldm21_project/train_fp.csv')
labels_pd_tp = pd.read_csv('/content/drive/MyDrive/mldm21_project/train_tp.csv')


пордок метод одиаков:

In [ ]:
np.sum(data['labels']!=labels['files_tp'])

0

Выбор меток которые соотвествуют TP с одним полезным сигналом, а так же преобразование выбор матриц-рисунков для этих меток

In [ ]:
lbl,cnt_lbl = np.unique(labels_pd_tp['recording_id'].values,return_counts=True)

In [ ]:
lbl_single = lbl[cnt_lbl==1]
length_1tp = len(lbl_single)
print(length_1tp)


1065


Извлечение данных

In [ ]:
mask = np.isin(data['labels'], lbl_single)

In [ ]:
data_to_nn = data['imgs'][mask].copy()
labels_to_nn = labels['labels_tp'][mask].copy()

In [ ]:
np.savez(PATH_TO_SAVE+'1signal_tp',data=data_to_nn,labels=labels_to_nn)

# 2. Исползование моделей

In [ ]:
import torchvision.models as models
import torch

import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms

#resnet18 = models.resnet18()
#alexnet = models.alexnet()
#vgg16 = models.vgg16()
#squeezenet = models.squeezenet1_0()
#densenet = models.densenet161()
#inception = models.inception_v3()
#googlenet = models.googlenet()
#shufflenet = models.shufflenet_v2_x1_0()
#mobilenet_v2 = models.mobilenet_v2()
#mobilenet_v3_large = models.mobilenet_v3_large()
#mobilenet_v3_small = models.mobilenet_v3_small()
#resnext50_32x4d = models.resnext50_32x4d()

In [ ]:
NUM_CLASSES = 3


num_classes = NUM_CLASSES


*   **resnet18,34,50**
*   **vgg16**
*   **alexnet**
*   **squeezenet**
*   **inception**
*  **mobilenet_v3_small**



In [ ]:
resnet18 = models.resnet18()
resnet18.fc = nn.Linear(512, num_classes)

resnet34 = models.resnet34()
resnet34.fc = nn.Linear(512, num_classes)

resnet50 = models.resnet50()
resnet50.fc = nn.Linear(2048, num_classes)


alexnet = models.alexnet()
alexnet.classifier[6] = nn.Linear(4096,num_classes)

vgg16 = models.vgg16()
vgg16.classifier[6] = nn.Linear(4096,num_classes)

squeezenet = models.squeezenet1_0()
squeezenet.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))

inception = models.inception_v3(aux_logits=False)  #'N x 3 x 299 x 299'
#inception.AuxLogits.fc = nn.Linear(768, num_classes)
inception.fc = nn.Linear(2048, num_classes)

mobilenet_v3_small = models.mobilenet_v3_small()
mobilenet_v3_small.classifier[0] = nn.Linear(in_features=576, out_features=4096, bias=True)
mobilenet_v3_small.classifier[3] = nn.Linear(4096,num_classes,bias=True)



/usr/local/lib/python3.7/dist-packages/torchvision/models/inception.py:83: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  ' due to scipy/scipy#11299), please set init_weights=True.', FutureWarning)


In [ ]:
model_stack = [resnet18,resnet34,resnet50, alexnet, vgg16, squeezenet,inception, mobilenet_v3_small ]
ModelNames = [str(imodel).split('(')[0] for imodel in model_stack]

Пример батча

In [ ]:
init = torch.Tensor(data_to_nn[0:4]).transpose(1,3)

for imodel in [resnet18,resnet34,resnet50,alexnet, vgg16, squeezenet,mobilenet_v3_small ]:
  ModelName = str(imodel).split('(')[0]
  print(ModelName) 
  print(imodel(init).size(),'\n')

ResNet
torch.Size([4, 3]) 

ResNet
torch.Size([4, 3]) 

ResNet
torch.Size([4, 3]) 

AlexNet
torch.Size([4, 3]) 

VGG
torch.Size([4, 3]) 

SqueezeNet
torch.Size([4, 3]) 

MobileNetV3
torch.Size([4, 3]) 



In [ ]:
from skimage.transform import resize
len_N = 4
init_incept = torch.Tensor(resize(data_to_nn[0:len_N],(len_N,299,299,3))).transpose(1,3)
init_incept.size()


torch.Size([4, 3, 299, 299])

In [ ]:
output = inception(init_incept)
torch.nn.functional.softmax(output, dim=0)


tensor([[0.1990, 0.3022, 0.3035],
        [0.3283, 0.3180, 0.2667],
        [0.2399, 0.1649, 0.2031],
        [0.2327, 0.2148, 0.2267]], grad_fn=<SoftmaxBackward0>)

# 3. Применение моделей

In [ ]:
import torchvision.models as models
import torch

import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms

NUM_CLASSES = 24

def crtclass(mdTp,num_classes=24):
  if mdTp == 'Res18':
    modelF = models.resnet18()
    modelF.fc = nn.Linear(512, num_classes)
  elif mdTp == 'Res34':
    modelF = models.resnet34()
    modelF.fc = nn.Linear(512, num_classes)
  elif mdTp == 'Res50':
    modelF = models.resnet50()
    modelF.fc = nn.Linear(2048, num_classes)
  elif mdTp == 'Alex':
    modelF = models.alexnet()
    modelF.classifier[6] = nn.Linear(4096,num_classes)
  elif mdTp == 'VGG':
    modelF = models.vgg16()
    modelF.classifier[6] = nn.Linear(4096,num_classes)
  elif mdTp == 'Squeez':
    modelF = models.squeezenet1_0()
    modelF.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
  elif mdTp == 'Modile_v3':
    modelF = models.mobilenet_v3_small()
    modelF.classifier[0] = nn.Linear(in_features=576, out_features=4096, bias=True)
    modelF.classifier[3] = nn.Linear(4096,num_classes,bias=True)
  else:
    assert False, 'no model'
  return modelF

In [ ]:
model  = crtclass('Res18')

In [ ]:
model(init)

tensor([[-0.8238, -0.9891,  0.1152,  0.8391,  0.1100,  0.4369,  0.1683, -0.1972,
         -0.1902, -0.1030,  1.3006, -0.3946,  0.4172,  0.2539,  0.1176,  0.4591,
         -0.0025, -0.1073,  0.5338, -0.2554,  0.5985,  0.7607,  0.5611, -0.7475],
        [-0.7351, -0.7711,  0.0317,  0.7263, -0.0838,  0.1344,  0.0955, -0.1218,
         -0.0711, -0.0356,  0.7619, -0.0765,  0.3333,  0.1243,  0.1694,  0.2714,
         -0.1433,  0.0771,  0.4070, -0.0550,  0.5891,  0.6991,  0.4625, -0.6502],
        [-0.7955, -0.8138,  0.0370,  0.7901,  0.1141,  0.3974,  0.0375, -0.2160,
         -0.1765, -0.0187,  1.0933, -0.2780,  0.5563,  0.1860,  0.0890,  0.4145,
         -0.0428,  0.0248,  0.5268, -0.1165,  0.4657,  0.7178,  0.5951, -0.8045],
        [-0.5852, -0.7955,  0.1387,  0.7437, -0.1796,  0.2990,  0.0609, -0.0935,
         -0.1755,  0.0277,  0.7812, -0.1039,  0.3799,  0.0659,  0.1737,  0.2862,
         -0.2631,  0.0210,  0.4816, -0.2182,  0.6061,  0.7948,  0.4368, -0.7184]],
       grad_fn=<AddmmBa